# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [25]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [26]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [27]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    EOS = target_vocab_to_int['<EOS>']
    
    source_id_text = []
    for sent in source_text.split('\n'):
        source_id_text.append([source_vocab_to_int[w] for w in sent.split()])
        
    target_id_text = []
    for sent in target_text.split('\n'):
        target_id_text.append([target_vocab_to_int[w] for w in sent.split()]+[EOS])
        
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1


/Users/qianweishuo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [31]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, shape=[None, None], name='input')
    target = tf.placeholder(tf.int32, shape=[None, None], name='target')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_seq_len = tf.placeholder(tf.int32, shape=[None], name='target_sequence_length')
    max_target_len = tf.reduce_max(target_seq_len, name='max_target_len')
    src_seq_len = tf.placeholder(tf.int32, shape=[None], name='source_sequence_length')
    
    return input, target, learning_rate, keep_prob, target_seq_len, max_target_len, src_seq_len


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/Users/qianweishuo/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/Users/qianweishuo/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/Users/qianweishuo/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>\n    app.launch_new_instance()', 'File "/Users/qianweishuo/anaconda3/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/Users/qianweishuo/anaconda3/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start\n    ioloop.IOLoop.instance().start()', 'File "/Users/qianweishuo/anaconda3/lib/py

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [32]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    x = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1])
    y = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), x], 1)
    return y
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [33]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.LSTMCell(rnn_size) for _ in range(num_layers)])
    drop = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
    encoder_outout, encoder_final_state = tf.nn.dynamic_rnn(drop, enc_embed_input, dtype=tf.float32)
    return encoder_outout, encoder_final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [34]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length)
    cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
#     cell = dec_cell
    decoder = tf.contrib.seq2seq.BasicDecoder(cell, helper, encoder_state, output_layer)
    training_decoder_output = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                                impute_finished=False,
                                                                maximum_iterations=max_summary_length)[0]
    return training_decoder_output

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [35]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id]), [batch_size], name='start_tokens')
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
#     cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob=keep_prob)
    cell = dec_cell
    decoder = tf.contrib.seq2seq.BasicDecoder(cell, helper, encoder_state, output_layer)
    inference_decoder_output  = tf.contrib.seq2seq.dynamic_decode(decoder,
                                                                  impute_finished=False,
                                                                  maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [36]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    # 2. output layer
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    
    # 3. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
#         cell = tf.contrib.rnn.DropoutWrapper(dec_cell, input_keep_prob=keep_prob)
        cell = dec_cell
        return cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    # 4. get outputs
    final_outputs_train = decoding_layer_train(encoder_state, 
                                         dec_cell,
                                         dec_embed_input,
                                         target_sequence_length, 
                                         max_target_sequence_length,
                                         output_layer,
                                         keep_prob)
    final_outputs_infer = decoding_layer_infer(encoder_state, 
                                              dec_cell,
                                              dec_embeddings,
                                              target_vocab_to_int['<GO>'],
                                              target_vocab_to_int['<EOS>'],
                                              max_target_sequence_length,
                                              target_vocab_size,
                                              output_layer,
                                              batch_size,
                                              keep_prob)
    return final_outputs_train, final_outputs_infer


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [38]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data,
                                  rnn_size,
                                  num_layers,
                                  keep_prob,
                                  source_sequence_length,
                                  source_vocab_size,
                                  enc_embedding_size)

    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)

    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(dec_input,
                                                                       enc_state,
                                                                       target_sequence_length,
                                                                       max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers,
                                                                       target_vocab_to_int,
                                                                       target_vocab_size,
                                                                       batch_size,
                                                                       keep_prob,
                                                                       dec_embedding_size)

    return training_decoder_output, inference_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [39]:
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.8
display_step = 10

### Build the Graph
Build the graph using the neural network you implemented.

In [40]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [41]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [42]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch   10/1077 - Train Accuracy: 0.1982, Validation Accuracy: 0.3050, Loss: 5.4466
Epoch   0 Batch   20/1077 - Train Accuracy: 0.2676, Validation Accuracy: 0.3349, Loss: 4.7164
Epoch   0 Batch   30/1077 - Train Accuracy: 0.2645, Validation Accuracy: 0.3356, Loss: 4.0809
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2660, Validation Accuracy: 0.3363, Loss: 3.6747
Epoch   0 Batch   50/1077 - Train Accuracy: 0.2652, Validation Accuracy: 0.3448, Loss: 3.4974
Epoch   0 Batch   60/1077 - Train Accuracy: 0.3013, Validation Accuracy: 0.3448, Loss: 3.2289
Epoch   0 Batch   70/1077 - Train Accuracy: 0.2410, Validation Accuracy: 0.3448, Loss: 3.3399
Epoch   0 Batch   80/1077 - Train Accuracy: 0.2840, Validation Accuracy: 0.3494, Loss: 3.1239
Epoch   0 Batch   90/1077 - Train Accuracy: 0.2938, Validation Accuracy: 0.3697, Loss: 3.0959
Epoch   0 Batch  100/1077 - Train Accuracy: 0.3172, Validation Accuracy: 0.3878, Loss: 3.0121
Epoch   0 Batch  110/1077 - Train Accuracy: 0.3723, Validati

Epoch   0 Batch  890/1077 - Train Accuracy: 0.5964, Validation Accuracy: 0.5636, Loss: 0.9530
Epoch   0 Batch  900/1077 - Train Accuracy: 0.5164, Validation Accuracy: 0.5536, Loss: 1.0695
Epoch   0 Batch  910/1077 - Train Accuracy: 0.5164, Validation Accuracy: 0.5593, Loss: 1.0197
Epoch   0 Batch  920/1077 - Train Accuracy: 0.5082, Validation Accuracy: 0.5621, Loss: 1.0296
Epoch   0 Batch  930/1077 - Train Accuracy: 0.5262, Validation Accuracy: 0.5533, Loss: 1.0052
Epoch   0 Batch  940/1077 - Train Accuracy: 0.5035, Validation Accuracy: 0.5568, Loss: 1.0281
Epoch   0 Batch  950/1077 - Train Accuracy: 0.5272, Validation Accuracy: 0.5639, Loss: 0.9776
Epoch   0 Batch  960/1077 - Train Accuracy: 0.5435, Validation Accuracy: 0.5568, Loss: 0.9360
Epoch   0 Batch  970/1077 - Train Accuracy: 0.5488, Validation Accuracy: 0.5611, Loss: 0.9909
Epoch   0 Batch  980/1077 - Train Accuracy: 0.5469, Validation Accuracy: 0.5586, Loss: 0.9638
Epoch   0 Batch  990/1077 - Train Accuracy: 0.5144, Validati

Epoch   1 Batch  700/1077 - Train Accuracy: 0.6035, Validation Accuracy: 0.6484, Loss: 0.6038
Epoch   1 Batch  710/1077 - Train Accuracy: 0.5840, Validation Accuracy: 0.6364, Loss: 0.6189
Epoch   1 Batch  720/1077 - Train Accuracy: 0.6377, Validation Accuracy: 0.6499, Loss: 0.6551
Epoch   1 Batch  730/1077 - Train Accuracy: 0.6121, Validation Accuracy: 0.6545, Loss: 0.6057
Epoch   1 Batch  740/1077 - Train Accuracy: 0.6270, Validation Accuracy: 0.6534, Loss: 0.6008
Epoch   1 Batch  750/1077 - Train Accuracy: 0.6457, Validation Accuracy: 0.6495, Loss: 0.5924
Epoch   1 Batch  760/1077 - Train Accuracy: 0.6465, Validation Accuracy: 0.6548, Loss: 0.6071
Epoch   1 Batch  770/1077 - Train Accuracy: 0.6455, Validation Accuracy: 0.6591, Loss: 0.5647
Epoch   1 Batch  780/1077 - Train Accuracy: 0.6238, Validation Accuracy: 0.6538, Loss: 0.6173
Epoch   1 Batch  790/1077 - Train Accuracy: 0.5805, Validation Accuracy: 0.6545, Loss: 0.6332
Epoch   1 Batch  800/1077 - Train Accuracy: 0.5922, Validati

Epoch   2 Batch  510/1077 - Train Accuracy: 0.7016, Validation Accuracy: 0.6978, Loss: 0.4247
Epoch   2 Batch  520/1077 - Train Accuracy: 0.7563, Validation Accuracy: 0.7113, Loss: 0.3911
Epoch   2 Batch  530/1077 - Train Accuracy: 0.6797, Validation Accuracy: 0.6999, Loss: 0.4384
Epoch   2 Batch  540/1077 - Train Accuracy: 0.7027, Validation Accuracy: 0.7006, Loss: 0.3905
Epoch   2 Batch  550/1077 - Train Accuracy: 0.7242, Validation Accuracy: 0.6985, Loss: 0.4377
Epoch   2 Batch  560/1077 - Train Accuracy: 0.7312, Validation Accuracy: 0.7198, Loss: 0.4077
Epoch   2 Batch  570/1077 - Train Accuracy: 0.6945, Validation Accuracy: 0.7006, Loss: 0.4525
Epoch   2 Batch  580/1077 - Train Accuracy: 0.7440, Validation Accuracy: 0.7085, Loss: 0.3826
Epoch   2 Batch  590/1077 - Train Accuracy: 0.6571, Validation Accuracy: 0.7049, Loss: 0.4460
Epoch   2 Batch  600/1077 - Train Accuracy: 0.7467, Validation Accuracy: 0.6875, Loss: 0.3956
Epoch   2 Batch  610/1077 - Train Accuracy: 0.7068, Validati

Epoch   3 Batch  320/1077 - Train Accuracy: 0.8234, Validation Accuracy: 0.8058, Loss: 0.3067
Epoch   3 Batch  330/1077 - Train Accuracy: 0.7895, Validation Accuracy: 0.7859, Loss: 0.2971
Epoch   3 Batch  340/1077 - Train Accuracy: 0.7973, Validation Accuracy: 0.7795, Loss: 0.2948
Epoch   3 Batch  350/1077 - Train Accuracy: 0.8277, Validation Accuracy: 0.8047, Loss: 0.3026
Epoch   3 Batch  360/1077 - Train Accuracy: 0.7934, Validation Accuracy: 0.7898, Loss: 0.2899
Epoch   3 Batch  370/1077 - Train Accuracy: 0.8032, Validation Accuracy: 0.7528, Loss: 0.2810
Epoch   3 Batch  380/1077 - Train Accuracy: 0.8039, Validation Accuracy: 0.7912, Loss: 0.2824
Epoch   3 Batch  390/1077 - Train Accuracy: 0.7562, Validation Accuracy: 0.7887, Loss: 0.3089
Epoch   3 Batch  400/1077 - Train Accuracy: 0.8398, Validation Accuracy: 0.7912, Loss: 0.2992
Epoch   3 Batch  410/1077 - Train Accuracy: 0.7800, Validation Accuracy: 0.7901, Loss: 0.3109
Epoch   3 Batch  420/1077 - Train Accuracy: 0.8441, Validati

...

Epoch  55 Batch 1000/1077 - Train Accuracy: 0.9847, Validation Accuracy: 0.9808, Loss: 0.0080
Epoch  55 Batch 1010/1077 - Train Accuracy: 0.9918, Validation Accuracy: 0.9826, Loss: 0.0061
Epoch  55 Batch 1020/1077 - Train Accuracy: 0.9984, Validation Accuracy: 0.9826, Loss: 0.0037
Epoch  55 Batch 1030/1077 - Train Accuracy: 0.9949, Validation Accuracy: 0.9808, Loss: 0.0073
Epoch  55 Batch 1040/1077 - Train Accuracy: 0.9848, Validation Accuracy: 0.9805, Loss: 0.0080
Epoch  55 Batch 1050/1077 - Train Accuracy: 0.9977, Validation Accuracy: 0.9805, Loss: 0.0034
Epoch  55 Batch 1060/1077 - Train Accuracy: 0.9906, Validation Accuracy: 0.9805, Loss: 0.0072
Epoch  55 Batch 1070/1077 - Train Accuracy: 0.9945, Validation Accuracy: 0.9858, Loss: 0.0049
Epoch  56 Batch   10/1077 - Train Accuracy: 0.9877, Validation Accuracy: 0.9808, Loss: 0.0072
Epoch  56 Batch   20/1077 - Train Accuracy: 0.9887, Validation Accuracy: 0.9798, Loss: 0.0064
Epoch  56 Batch   30/1077 - Train Accuracy: 0.9914, Validati

Epoch  56 Batch  810/1077 - Train Accuracy: 0.9952, Validation Accuracy: 0.9780, Loss: 0.0035
Epoch  56 Batch  820/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9783, Loss: 0.0035
Epoch  56 Batch  830/1077 - Train Accuracy: 0.9891, Validation Accuracy: 0.9783, Loss: 0.0054
Epoch  56 Batch  840/1077 - Train Accuracy: 0.9934, Validation Accuracy: 0.9805, Loss: 0.0062
Epoch  56 Batch  850/1077 - Train Accuracy: 0.9900, Validation Accuracy: 0.9783, Loss: 0.0083
Epoch  56 Batch  860/1077 - Train Accuracy: 0.9967, Validation Accuracy: 0.9833, Loss: 0.0063
Epoch  56 Batch  870/1077 - Train Accuracy: 0.9996, Validation Accuracy: 0.9822, Loss: 0.0065
Epoch  56 Batch  880/1077 - Train Accuracy: 0.9902, Validation Accuracy: 0.9819, Loss: 0.0080
Epoch  56 Batch  890/1077 - Train Accuracy: 0.9844, Validation Accuracy: 0.9798, Loss: 0.0070
Epoch  56 Batch  900/1077 - Train Accuracy: 0.9816, Validation Accuracy: 0.9822, Loss: 0.0069
Epoch  56 Batch  910/1077 - Train Accuracy: 0.9885, Validati

Epoch  57 Batch  620/1077 - Train Accuracy: 0.9789, Validation Accuracy: 0.9826, Loss: 0.0078
Epoch  57 Batch  630/1077 - Train Accuracy: 0.9891, Validation Accuracy: 0.9872, Loss: 0.0070
Epoch  57 Batch  640/1077 - Train Accuracy: 0.9959, Validation Accuracy: 0.9869, Loss: 0.0045
Epoch  57 Batch  650/1077 - Train Accuracy: 0.9930, Validation Accuracy: 0.9780, Loss: 0.0047
Epoch  57 Batch  660/1077 - Train Accuracy: 0.9941, Validation Accuracy: 0.9872, Loss: 0.0045
Epoch  57 Batch  670/1077 - Train Accuracy: 0.9904, Validation Accuracy: 0.9822, Loss: 0.0055
Epoch  57 Batch  680/1077 - Train Accuracy: 0.9829, Validation Accuracy: 0.9830, Loss: 0.0057
Epoch  57 Batch  690/1077 - Train Accuracy: 0.9906, Validation Accuracy: 0.9872, Loss: 0.0086
Epoch  57 Batch  700/1077 - Train Accuracy: 0.9938, Validation Accuracy: 0.9872, Loss: 0.0036
Epoch  57 Batch  710/1077 - Train Accuracy: 0.9949, Validation Accuracy: 0.9872, Loss: 0.0046
Epoch  57 Batch  720/1077 - Train Accuracy: 0.9918, Validati

Epoch  58 Batch  430/1077 - Train Accuracy: 0.9934, Validation Accuracy: 0.9755, Loss: 0.0057
Epoch  58 Batch  440/1077 - Train Accuracy: 0.9859, Validation Accuracy: 0.9805, Loss: 0.0062
Epoch  58 Batch  450/1077 - Train Accuracy: 0.9715, Validation Accuracy: 0.9730, Loss: 0.0080
Epoch  58 Batch  460/1077 - Train Accuracy: 0.9918, Validation Accuracy: 0.9727, Loss: 0.0066
Epoch  58 Batch  470/1077 - Train Accuracy: 0.9934, Validation Accuracy: 0.9755, Loss: 0.0056
Epoch  58 Batch  480/1077 - Train Accuracy: 0.9807, Validation Accuracy: 0.9759, Loss: 0.0112
Epoch  58 Batch  490/1077 - Train Accuracy: 0.9891, Validation Accuracy: 0.9801, Loss: 0.0066
Epoch  58 Batch  500/1077 - Train Accuracy: 0.9898, Validation Accuracy: 0.9822, Loss: 0.0037
Epoch  58 Batch  510/1077 - Train Accuracy: 0.9906, Validation Accuracy: 0.9794, Loss: 0.0059
Epoch  58 Batch  520/1077 - Train Accuracy: 0.9862, Validation Accuracy: 0.9805, Loss: 0.0042
Epoch  58 Batch  530/1077 - Train Accuracy: 0.9867, Validati

Epoch  59 Batch  240/1077 - Train Accuracy: 1.0000, Validation Accuracy: 0.9808, Loss: 0.0035
Epoch  59 Batch  250/1077 - Train Accuracy: 0.9798, Validation Accuracy: 0.9826, Loss: 0.0066
Epoch  59 Batch  260/1077 - Train Accuracy: 0.9944, Validation Accuracy: 0.9826, Loss: 0.0042
Epoch  59 Batch  270/1077 - Train Accuracy: 0.9711, Validation Accuracy: 0.9801, Loss: 0.0154
Epoch  59 Batch  280/1077 - Train Accuracy: 0.9867, Validation Accuracy: 0.9801, Loss: 0.0086
Epoch  59 Batch  290/1077 - Train Accuracy: 0.9832, Validation Accuracy: 0.9897, Loss: 0.0115
Epoch  59 Batch  300/1077 - Train Accuracy: 0.9860, Validation Accuracy: 0.9847, Loss: 0.0061
Epoch  59 Batch  310/1077 - Train Accuracy: 0.9945, Validation Accuracy: 0.9808, Loss: 0.0053
Epoch  59 Batch  320/1077 - Train Accuracy: 0.9840, Validation Accuracy: 0.9808, Loss: 0.0076
Epoch  59 Batch  330/1077 - Train Accuracy: 0.9922, Validation Accuracy: 0.9794, Loss: 0.0058
Epoch  59 Batch  340/1077 - Train Accuracy: 0.9971, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [43]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [44]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [45]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    unk_id = vocab_to_int['<UNK>']
    return [vocab_to_int.get(w, unk_id) for w in sentence.lower().split()]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [46]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [40, 87, 204, 41, 230, 11, 7]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [40, 126, 216, 126, 216, 216, 16, 263, 281, 1]
  French Words: il au volant au volant volant vieux brillant . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.